In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [74]:
df = pd.read_csv("Data/marketing_campaign.csv", sep="\t")
drop_columns = ["ID", 'Z_CostContact', 'Z_Revenue']
df = df.drop(drop_columns, axis=1)
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,...,10,4,7,0,0,0,0,0,0,1
1,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,...,1,2,5,0,0,0,0,0,0,0
2,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,...,2,10,4,0,0,0,0,0,0,0
3,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,...,0,4,6,0,0,0,0,0,0,0
4,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,...,3,6,5,0,0,0,0,0,0,0


In [75]:
from datetime import datetime

for row in range(len(df["Dt_Customer"])):
  df["Dt_Customer"][row] = datetime.strptime(df["Dt_Customer"][row], '%d-%m-%Y').year 
  
df.head()

/tmp/ipykernel_31045/3656975255.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Dt_Customer"][row] = datetime.strptime(df["Dt_Customer"][row], '%d-%m-%Y').year


,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,1957,Graduation,Single,58138.0,0,0,2012,58,635,88,...,10,4,7,0,0,0,0,0,0,1
1,1954,Graduation,Single,46344.0,1,1,2014,38,11,1,...,1,2,5,0,0,0,0,0,0,0
2,1965,Graduation,Together,71613.0,0,0,2013,26,426,49,...,2,10,4,0,0,0,0,0,0,0
3,1984,Graduation,Together,26646.0,1,0,2014,26,11,4,...,0,4,6,0,0,0,0,0,0,0
4,1981,PhD,Married,58293.0,1,0,2014,94,173,43,...,3,6,5,0,0,0,0,0,0,0


In [76]:
nominal_at = ['Education', 'Marital_Status', 'Dt_Customer']
le = LabelEncoder()

for atribute in nominal_at:
  df[atribute] = le.fit_transform(df[atribute])

df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,1957,2,4,58138.0,0,0,0,58,635,88,...,10,4,7,0,0,0,0,0,0,1
1,1954,2,4,46344.0,1,1,2,38,11,1,...,1,2,5,0,0,0,0,0,0,0
2,1965,2,5,71613.0,0,0,1,26,426,49,...,2,10,4,0,0,0,0,0,0,0
3,1984,2,5,26646.0,1,0,2,26,11,4,...,0,4,6,0,0,0,0,0,0,0
4,1981,4,3,58293.0,1,0,2,94,173,43,...,3,6,5,0,0,0,0,0,0,0


In [77]:
scaler = StandardScaler()
for column in df.columns:
  df[column] = scaler.fit_transform(df[[column]]).flatten()

df.head()
df.describe()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
count,2.240000e+03,2.240000e+03,2.240000e+03,2.216000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,...,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03
mean,-8.133970e-15,1.173664e-16,-1.379849e-16,4.889791e-17,-1.586033e-17,-3.172066e-18,7.652609e-17,-3.172066e-18,-7.612958e-17,-2.379049e-17,...,9.516197e-18,7.612958e-17,-6.344132e-17,-2.854859e-17,5.551115e-17,-3.013462e-17,6.264830e-17,2.696256e-17,-3.053113e-17,-7.295751e-17
std,1.000223e+00,1.000223e+00,1.000223e+00,1.000226e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,...,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00
min,-6.326960e+00,-2.128637e+00,-3.466341e+00,-2.007250e+00,-8.252176e-01,-9.298944e-01,-1.502225e+00,-1.696001e+00,-9.031668e-01,-6.614492e-01,...,-9.108985e-01,-1.781466e+00,-2.191381e+00,-2.801403e-01,-2.838302e-01,-2.801403e-01,-2.621112e-01,-1.165103e-01,-9.728167e-02,-4.186121e-01
25%,-8.184192e-01,-3.501413e-01,-6.783324e-01,-6.732620e-01,-8.252176e-01,-9.298944e-01,-4.109430e-02,-8.671566e-01,-8.325919e-01,-6.363012e-01,...,-9.108985e-01,-8.584551e-01,-9.548307e-01,-2.801403e-01,-2.838302e-01,-2.801403e-01,-2.621112e-01,-1.165103e-01,-9.728167e-02,-4.186121e-01
50%,9.967091e-02,-3.501413e-01,2.510037e-01,-3.439972e-02,-8.252176e-01,-9.298944e-01,-4.109430e-02,-3.777284e-03,-3.875991e-01,-4.602650e-01,...,-2.265407e-01,-2.431145e-01,2.817201e-01,-2.801403e-01,-2.838302e-01,-2.801403e-01,-2.621112e-01,-1.165103e-01,-9.728167e-02,-4.186121e-01
75%,6.839101e-01,5.391064e-01,1.180340e+00,6.466600e-01,1.032559e+00,9.069340e-01,-4.109430e-02,8.596020e-01,5.952483e-01,1.684356e-01,...,4.578170e-01,6.798964e-01,6.939037e-01,-2.801403e-01,-2.838302e-01,-2.801403e-01,-2.621112e-01,-1.165103e-01,-9.728167e-02,-4.186121e-01
max,2.269702e+00,1.428354e+00,3.039012e+00,2.441328e+01,2.890335e+00,2.743762e+00,1.420036e+00,1.722981e+00,3.533390e+00,4.343008e+00,...,8.670110e+00,2.218248e+00,6.052291e+00,3.569640e+00,3.523233e+00,3.569640e+00,3.815174e+00,8.582929e+00,1.027943e+01,2.388846e+00
